# Specific the region lables.
### CTV 
168 - 174, 187 - 195, 207 - 215, 228 - 234.    
every voxel receives a uniform dose of 82.8 Gy    
PS: 95% - 105% range = 78.66 to 86.94 Gy, considering cold (low) and hot (high) limitation, set the uniform range to [79.1,86.94] Gy.
### Bladder 
88 - 94, 107 - 115, 127 - 135, 148 - 154.  
max dose to a voxel: 81.0 Gy. 
average dose should be <= 50.0 Gy  
at most 10% of the bladder should receive a dose > 65.0 Gy
### Rectum
249 - 253, 268 - 274, 288 - 294, 309 - 313.   
max dose to a voxel: 79.2 Gy  
average dose should be <= 40.0 Gy
### Left femur head 
117 - 118, 136 - 139, 156 - 159, 176 - 179, 196 - 199, 216 - 219, 236 - 239, 257 - 258.   
max dose to a voxel: 50.0 Gy  
At most 15% of the left femur head should receive > 40.0 Gy
### Right femur head 
103 - 104, 122 - 125, 142 - 145, 162 - 165, 182 - 185, 202 - 205, 222 - 225, 243 - 244.    
max dose to a voxel: 50.0 Gy   
At most 15% of the right femur head should receive > 40.0 Gy    
### Unspecified   
max dose to a voxel: 72.0 Gy

In [448]:
from ortools.linear_solver import pywraplp
import numpy as np
import pandas as pd
import math

In [449]:
#Loading data
#set variables
df = pd.read_excel("DoseMatrix.xlsx")
variables = [0 for i in range(60)]
y = [0 for i in range(400)]
dose = df.iloc[:,1:].to_numpy()
solver = pywraplp.Solver.CreateSolver('final project', 'GLOP')
objective=solver.Objective()
objective.SetMinimization()
#add variables
total_dose = []
for i in range(60):
    variables[i] = solver.IntVar(0, solver.infinity(), "x_"+str(i))
    for j in range(400): 
        total_dose.append(variables[i]* dose[j][i])
for i in range(400):
    y[i] = solver.IntVar(0, 1, "y_"+str(i))
#set objective function: to maximize total dose within the constrain
solver.Minimize(sum(total_dose))

In [450]:
# CTV 167 - 173, 186 - 194, 204 - 214, 227 - 233.
# Bladder 87 - 93, 106 - 114, 126 - 134, 147 - 153.
# Rectum 248 - 252, 267 - 273, 287 - 293, 308 - 312.
# Left femur head 116 - 117, 135 - 138, 155 - 158, 175 - 178, 195 - 198, 215 - 218, 235 - 238, 256 - 257.
# Right femur head 102 - 103, 121 - 124, 141 - 144, 161 - 164, 181 - 184, 201 - 204, 221 - 224, 242 - 243.
# print(list(range(400)))
ctv = [167, 168, 169, 170, 171, 172, 173, 186, 187, 188, 189, 190, 191, 192, 193, 194, 207, 208, 209, 210, 211, 212, 213, 214, 227, 228, 229, 230, 231, 232, 233]
bladder = [87, 88, 89, 90, 91, 92, 93, 106, 107, 108, 109, 110, 111, 112, 113, 114, 126, 127, 128, 129, 130, 131, 132, 133, 134, 147, 148, 149, 150, 151, 152, 153]
rectum = [248, 249, 250, 251, 252, 267, 268, 269, 270, 271, 272, 273, 287, 288, 289, 290, 291, 292, 293, 308, 309, 310, 311, 312, 312]
left = [116, 117, 135, 136, 137, 138, 155, 156, 157, 158, 175, 176, 177, 178, 195, 196, 197, 198, 215, 216, 217, 218, 235, 236, 237, 238, 256, 257]
right = [102, 103, 121, 122, 123, 124, 141, 142, 143, 144, 161, 162, 163, 164, 181, 182, 183, 184, 201, 202, 203, 204, 221, 222, 223, 224, 242, 243]
union = list(set(ctv).union(set(bladder)).union(set(rectum).union(set(left).union(set(right)))))
unspecific = list(set(list(range(400))).difference(set(union)))

In [451]:
#Plan A: Only care abou the CTV, Left, right and unspecific region
#CTV
total_ctv = []
for i in ctv:
    voxel_ctv = []
    for j in range(60):
        voxel_ctv.append(variables[j]* dose[i][j])
    total_ctv.append(sum(voxel_ctv)) 
for z in range (len(total_ctv)):
    solver.Add(total_ctv[z] <= 86.94)
    solver.Add(total_ctv[z] >= 79.1)
        
#Bladder
total_bladder = []
y_bladder = []
for i in bladder:
    voxel_bladder = []
    for j in range(60):
        voxel_bladder.append(variables[j]* dose[i][j])
    total_bladder.append(sum(voxel_bladder))
    y_bladder.append(y[i])
solver.Add(sum(y_bladder) <= math.floor(len(bladder)*0.1))
solver.Add(np.mean(total_bladder) <= 50)#50
for z in range (len(total_bladder)):
    solver.Add(total_bladder[z] <= 81) #81
    solver.Add(total_bladder[z] <= 65 + 20 * y_bladder[z]) #65
    

#rectum
total_rectum = []
for i in rectum:
    voxel_rectum = []
    for j in range(60):
        voxel_rectum.append(variables[j]* dose[i][j])
    total_rectum.append(sum(voxel_rectum)) 
solver.Add(np.mean(total_rectum) <= 40)#40
for z in range (len(total_rectum)):    
    solver.Add(total_rectum[z] <= 79.2)#79.2 

#left
greater_left = []
total_left = []
y_left = []
for i in left:
    voxel_left = []
    for j in range(60):
        voxel_left.append(variables[j]* dose[i][j])
    y_left.append(y[i])
    total_left.append(sum(voxel_left)) 
solver.Add(sum(y_left) <=math.floor(len(right)*0.15))
for z in range (len(total_left)):    
    solver.Add(total_left[z] <= 51) 
    solver.Add(total_left[z] <= 40 + 20 * y_left[z])

#right
greater_right = []
total_right = []
y_right = []
for i in right:
    voxel_right = []
    for j in range(60):
        voxel_right.append(variables[j]* dose[i][j])
    y_right.append(y[i])
    total_right.append(sum(voxel_right))
solver.Add(sum(y_right) <=math.floor(len(right)*0.15))
for z in range (len(total_right)):    
    solver.Add(total_right[z] <= 51) 
    solver.Add(total_right[z] <= 40 + 20 * y_right[z])

#unspecific
total_unspecific = []
for i in unspecific:
    voxel_unspecific = []
    for j in range(60):
        voxel_unspecific.append(variables[j]* dose[i][j])
    total_unspecific.append(sum(voxel_unspecific)) 
for z in range (len(total_unspecific)):
    solver.Add(total_unspecific[z] <= 80) #72

In [452]:
#Plan A: Only care abou the CTV, Left, right and unspecific region
#CTV
total_ctv = []
for i in ctv:
    voxel_ctv = []
    for j in range(60):
        voxel_ctv.append(variables[j]* dose[i][j])
    total_ctv.append(sum(voxel_ctv)) 
for z in range (len(total_ctv)):
    solver.Add(total_ctv[z] <= 86.94)
    solver.Add(total_ctv[z] >= 79.1)
        
#Bladder
total_bladder = []
y_bladder = []
for i in bladder:
    voxel_bladder = []
    for j in range(60):
        voxel_bladder.append(variables[j]* dose[i][j])
    total_bladder.append(sum(voxel_bladder))
    y_bladder.append(y[i])
solver.Add(sum(y_bladder) <= math.floor(len(bladder)*0.1))
solver.Add(np.mean(total_bladder) <= 70)#50
for z in range (len(total_bladder)):
    solver.Add(total_bladder[z] <= 82) #81
#     solver.Add(total_bladder[z] <= 65 + 20 * y_bladder[z]) #65
    

#rectum
total_rectum = []
for i in rectum:
    voxel_rectum = []
    for j in range(60):
        voxel_rectum.append(variables[j]* dose[i][j])
    total_rectum.append(sum(voxel_rectum)) 
solver.Add(np.mean(total_rectum) <= 78)#40
for z in range (len(total_rectum)):    
    solver.Add(total_rectum[z] <= 84)#79.2 

#left
greater_left = []
total_left = []
y_left = []
for i in left:
    voxel_left = []
    for j in range(60):
        voxel_left.append(variables[j]* dose[i][j])
    y_left.append(y[i])
    total_left.append(sum(voxel_left)) 
solver.Add(sum(y_left) <=math.floor(len(right)*0.15))
for z in range (len(total_left)):    
    solver.Add(total_left[z] <= 52) #50
    solver.Add(total_left[z] <= 40 + 20 * y_left[z])

#right
greater_right = []
total_right = []
y_right = []
for i in right:
    voxel_right = []
    for j in range(60):
        voxel_right.append(variables[j]* dose[i][j])
    y_right.append(y[i])
    total_right.append(sum(voxel_right))
solver.Add(sum(y_right) <=math.floor(len(right)*0.15))
for z in range (len(total_right)):    
    solver.Add(total_right[z] <= 52) #50
    solver.Add(total_right[z] <= 40 + 20 * y_right[z])

#unspecific
total_unspecific = []
for i in unspecific:
    voxel_unspecific = []
    for j in range(60):
        voxel_unspecific.append(variables[j]* dose[i][j])
    total_unspecific.append(sum(voxel_unspecific)) 
for z in range (len(total_unspecific)):
    solver.Add(total_unspecific[z] <= 90) #72

In [453]:
#run the solver
status = solver.Solve()
#find the optimal solution by solver
if status == solver.OPTIMAL:
    print("Solver reaches the optimal solution with objective function = " + str(objective.Value()))

else:
    print('Solver cannot find optimal solution')


Solver reaches the optimal solution with objective function = 18591.74632208237


In [457]:
x = np.zeros(60)
for i in range(60):
    x[i] = math.floor(variables[i].solution_value())
print(x)

[ 0. 22. 42.  0.  0. 71.  0.  0. 54. 17.  0.  0.  0.  0.  0.  0.  0. 11.
  4.  0.  0.  1. 16.  0.  0.  0.  0.  0.  0.  0. 42. 12.  0.  0. 20.  0.
  0.  0. 26.  0.  0.  0.  8.  0.  8.  6. 26.  0.  0.  0.  0.  0. 37.  0.
 27.  5.  0.  0.  0.  0.]
